In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Данные о 1115 магазинах. Целевая переменная - Sales.

Если магазин закрыт, он не получает прибыли.

Длительные акции начинаются в начале, середине или конце сезона, или никогда не начинаются.

In [ ]:
df_store = pd.read_csv('store.csv', index_col='Store')

In [ ]:
df_store.info()

In [ ]:
print df_store.shape
df_store.head()

In [ ]:
df_store.describe()

In [ ]:
for c in df_store.columns:
    print "\n{0}: range [{1}, {2}]".format(c, df_store[c].min(), df_store[c].max())
    if df_store[c].unique().shape[0] < 10:
        print "uniques: {}".format(df_store[c].unique())

Категориальные признаки: StoreType, Assortment, Promo2 (bin!), PromoInterval

Численные непрерывные: CompetitionDistance

Численные дискретные: CompetitionOpenSinceMonth, CompetitionOpenSinceYear, Promo2SinceWeek, Promo2SinceWeek

In [ ]:
categorial_features = ['StoreType', 'Assortment', 'PromoInterval']
numerical_features = ['']

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

for p in categorial_features:
    X_int = LabelEncoder().fit_transform(df_store[p].values.reshape(-1, 1)).reshape(-1,1)
    ohe_feat = OneHotEncoder(sparse=False).fit_transform(X_int)
    tmp = pd.DataFrame(ohe_feat, columns=['{0}='.format(p) + str(i) for i in df_store[p].unique()], 
                       index=df_store.index,
                       dtype=int)
    df_store = pd.concat([df_store, tmp], axis=1)
    df_store = df_store.drop(p, axis=1)

In [ ]:
df_store.head()

In [ ]:
comp_feats = ['CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']
promo_feats = ['Promo2SinceWeek', 'Promo2SinceYear']

In [ ]:
df_store_YCYP = df_store[~pd.isnull(df_store.CompetitionDistance) & (df_store.Promo2 == 1)]
df_store_YCNP = df_store[~pd.isnull(df_store.CompetitionDistance) & (df_store.Promo2 == 0)].drop(promo_feats, axis=1)
df_store_NCYP = df_store[pd.isnull(df_store.CompetitionDistance) & (df_store.Promo2 == 1)].drop(comp_feats, axis=1)
df_store_NCNP = df_store[pd.isnull(df_store.CompetitionDistance) & (df_store.Promo2 == 0)].drop(comp_feats + promo_feats, axis=1)
print df_store_YCYP.shape, df_store_YCNP.shape, df_store_NCYP.shape, df_store_NCNP.shape

Т.о., практически у всех аптек есть конкуренты, а вот наличие акций делится примерно поровну. Теперь у нас есть четыре датафрейма, полностью наполненные числовыми параметрами.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

scores = []
ns = range(2, 10) + range(10, 90, 5)
for n in ns:
    agc = AgglomerativeClustering(n_clusters=n)
#store_cluster = agc.fit_predict(df_store.drop(['Promo2SinceWeek', 'Promo2SinceYear'], axis=1).dropna()).reshape(-1, 1)
#store_cluster = agc.fit_predict(df_store.drop(
#    ['CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',
#     'Promo2SinceWeek', 'Promo2SinceYear'],
#    axis=1).dropna()).reshape(-1, 1)
    store_cluster = agc.fit_predict(df_store.fillna(0)).reshape(-1,1)
    scores.append(silhouette_score(df_store.fillna(0), store_cluster.ravel()))
plt.plot(ns, scores)

In [ ]:
agc = AgglomerativeClustering(n_clusters=40)
store_cluster = agc.fit_predict(df_store.fillna(0))
store_cluster.shape

In [ ]:
df_train = pd.read_csv('train.csv', index_col='Store')

In [ ]:
print df_train.shape
df_train.head()

In [ ]:
df_train.StateHoliday.unique()

In [ ]:
df_train.replace({'StateHoliday': {0: '0'}}, inplace=True)
df_train.StateHoliday.unique()

In [ ]:
SH_int = LabelEncoder().fit_transform(df_train.StateHoliday.values.reshape(-1, 1)).reshape(-1,1)
ohe_feat = OneHotEncoder(sparse=False).fit_transform(SH_int)
tmp = pd.DataFrame(ohe_feat, columns=['SH='+ str(i) for i in df_train.StateHoliday.unique()], 
                   index=df_train.index,
                   dtype=int)
df_train = df_train.drop('StateHoliday', axis=1)
df_train = pd.concat([df_train, tmp], axis=1)

In [ ]:
for p in ['DayOfWeek']:
    ohe_feat = OneHotEncoder(sparse=False).fit_transform(df_train[p].values.reshape(-1,1))
    tmp = pd.DataFrame(ohe_feat, columns=['{}='.format(p) + str(i) for i in df_train[p].unique()], 
                       index=df_train.index,
                       dtype=int)
    df_train = df_train.drop(p, axis=1)
    df_train = pd.concat([df_train, tmp], axis=1)

In [ ]:
df_train['label'] = pd.Series([store_cluster[ind - 1] for ind in df_train.index],
                              index=df_train.index)

y_train = df_train.Sales.values
X_train = df_train.drop(['Date', 'Sales', 'Customers'], axis=1).values

In [ ]:
df_train.head()

In [ ]:
X_train[:5]

In [ ]:
## потом окрасить это в цвета кластеров аптек
plt.figure(figsize=(10,10))
plt.scatter(df_train.Customers, df_train.Sales, c=df_train.label)

In [ ]:
f, axs = plt.subplots(10, 4, sharex=True, sharey=True)
f.set_figheight(15)
f.set_figwidth(15)
for i, ax in enumerate(axs):
    for j, axj in enumerate(ax):
        tmp = df_train[df_train.label == (4 * i + j)]
        axj.scatter(tmp.Customers, tmp.Sales)
        axj.set_ylim(0, 50000)
        axj.set_xlim(0, 8000)

Можно предположить, что в пределах кластера зависимость прибыли от числа покупателей практически линейна, а число покупателей определяется другими параметрами. Следовательно, можно попробовать предсказать прибыль по линейной регрессии.

In [ ]:
# здесь могла быть ваша линейная регрессия
from sklearn.linear_model import LinearRegression, Ridge

#lnr = LinearRegression(n_jobs=-1)
#lnr.fit(X_train, y_train)

rlnr = Ridge()
rlnr.fit(X_train, y_train)

In [ ]:
rlnr.coef_, rlnr.intercept_

In [ ]:
df_test = pd.read_csv('test.csv', index_col='Id')

In [ ]:
print df_test.shape
df_test.head()

In [ ]:
df_test.replace({'StateHoliday': {0: '0'}}, inplace=True)

SH_int = LabelEncoder().fit_transform(df_test.StateHoliday.values.reshape(-1, 1)).reshape(-1,1)
ohe_feat = OneHotEncoder(sparse=False).fit_transform(SH_int)
tmp = pd.DataFrame(ohe_feat, columns=['SH='+ str(i) for i in df_test.StateHoliday.unique()], 
                   index=df_test.index,
                   dtype=int)
df_test = df_test.drop('StateHoliday', axis=1)
df_test = pd.concat([df_test, tmp], axis=1)
df_test['SH=b'] = 0
df_test['SH=c'] = 0

for p in ['DayOfWeek']:
    ohe_feat = OneHotEncoder(sparse=False).fit_transform(df_test[p].values.reshape(-1,1))
    tmp = pd.DataFrame(ohe_feat, columns=['{}='.format(p) + str(i) for i in df_test[p].unique()], 
                       index=df_test.index,
                       dtype=int)
    df_test = df_test.drop(p, axis=1)
    df_test = pd.concat([df_test, tmp], axis=1)
    
df_test['label'] = pd.Series([store_cluster[ind - 1] for ind in df_test.Store],
                              index=df_test.index)

df_test = df_test.fillna(0)

X_test = df_test.drop(['Store', 'Date'], axis=1).values

In [ ]:
df_test.head()

In [ ]:
y_pred = rlnr.predict(X_test)

In [ ]:
y_pred.mean(), y_pred.min(), y_pred.max()

In [ ]:
out = np.concatenate((np.arange(1, y_pred.shape[0]+1).reshape(-1,1), y_pred.reshape(-1,1)), axis=1)
np.savetxt('pred.csv', out, delimiter=',', fmt=['%d', '%f'], header='"Id","Sales"')